In [10]:
import pandas as pd
import numpy as np

# Cargar datos
file_path = "../data/nasa.csv"
df = pd.read_csv(file_path, low_memory=False)
print("Datos cargados. Tamaño inicial:", df.shape)

Datos cargados. Tamaño inicial: (38095, 288)


In [11]:

# Seleccionar columnas clave para la habitabilidad
relevant_columns = ['pl_name', 'hostname', 'pl_orbper', 'pl_orbsmax', 'pl_orbeccen',
                    'pl_rade', 'pl_bmasse', 'pl_insol', 'pl_eqt',
                    'st_teff', 'st_rad', 'st_mass', 'st_met', 'discoverymethod', 'disc_year', 'pl_refname']
df = df[relevant_columns]
print("Selección de columnas clave completada. Tamaño actual:", df.shape)

Selección de columnas clave completada. Tamaño actual: (38095, 16)


In [12]:

# Mostrar ejemplos antes de la unificación
print("Ejemplos antes de la unificación:")
print(df[df['pl_name'].duplicated(keep=False)].head(6))

# Agrupar por el nombre del exoplaneta para unificar datos duplicados
def merge_numeric(series):
    return series.dropna().mean() if not series.dropna().empty else np.nan

def merge_categorical(series):
    return ', '.join(series.dropna().unique()) if not series.dropna().empty else np.nan

aggregation_functions = {
    'hostname': merge_categorical,
    'pl_orbper': merge_numeric,
    'pl_orbsmax': merge_numeric,
    'pl_orbeccen': merge_numeric,
    'pl_rade': merge_numeric,
    'pl_bmasse': merge_numeric,
    'pl_insol': merge_numeric,
    'pl_eqt': merge_numeric,
    'st_teff': merge_numeric,
    'st_rad': merge_numeric,
    'st_mass': merge_numeric,
    'st_met': merge_numeric,
    'discoverymethod': merge_categorical,
    'disc_year': 'max',  # Mantener el año más reciente
    'pl_refname': merge_categorical
}

df = df.groupby('pl_name', as_index=False).agg(aggregation_functions)
print("Unificación de registros completada. Tamaño actual:", df.shape)

# Mostrar ejemplos después de la unificación
print("Ejemplos después de la unificación:")
print(df.head(3))

Ejemplos antes de la unificación:
    pl_name hostname  pl_orbper  pl_orbsmax  pl_orbeccen  pl_rade   pl_bmasse  \
0  11 Com b   11 Com  326.03000       1.290        0.231      NaN  6165.60000   
1  11 Com b   11 Com        NaN       1.210          NaN      NaN  5434.70000   
2  11 Com b   11 Com  323.21000       1.178        0.238      NaN  4914.89849   
3  11 UMi b   11 UMi  516.21997       1.530        0.080      NaN  4684.81420   
4  11 UMi b   11 UMi  516.22000       1.540        0.080      NaN  3337.07000   
5  11 UMi b   11 UMi        NaN       1.510          NaN      NaN  3432.40000   

   pl_insol  pl_eqt  st_teff  st_rad  st_mass  st_met  discoverymethod  \
0       NaN     NaN   4742.0   19.00     2.70   -0.35  Radial Velocity   
1       NaN     NaN      NaN     NaN     2.60     NaN  Radial Velocity   
2       NaN     NaN   4874.0   13.76     2.09   -0.26  Radial Velocity   
3       NaN     NaN   4213.0   29.79     2.78   -0.02  Radial Velocity   
4       NaN     NaN   4340.0

In [13]:

# Filtrar exoplanetas con datos clave
required_columns = ['pl_bmasse', 'pl_eqt', 'pl_insol', 'pl_orbsmax', 'st_teff', 'st_rad']
df = df.dropna(subset=required_columns)
print("Filtrado de exoplanetas con datos esenciales completado. Tamaño actual:", df.shape)

# Mostrar información completa de algunos planetas tras encontrar planetas con info completa
print("Ejemplos de planetas con información completa tras encontrar planetas con info completa:")
print(df.sample(3))

Filtrado de exoplanetas con datos esenciales completado. Tamaño actual: (1382, 16)
Ejemplos de planetas con información completa tras encontrar planetas con info completa:
         pl_name  hostname  pl_orbper  pl_orbsmax  pl_orbeccen   pl_rade  \
5084   TOI-125 d   TOI-125  19.980785    0.137000        0.168   2.73850   
5437   TOI-519 b   TOI-519   1.265233    0.014793        0.189  11.83325   
5228  TOI-2015 b  TOI-2015   3.349054    0.030100        0.000   4.38350   

        pl_bmasse  pl_insol  pl_eqt  st_teff  st_rad  st_mass  st_met  \
5084    13.600000    21.645   600.0  5237.00   0.850     0.86 -0.0200   
5437  1587.878153    53.995   723.0  3312.75   0.360     0.36  0.2675   
5228    16.400000    21.245   584.5  3208.50   0.325     0.34  0.0500   

     discoverymethod  disc_year  \
5084         Transit       2020   
5437         Transit       2021   
5228         Transit       2023   

                                             pl_refname  
5084  <a refstr=EXOFOP_TESS_TOI

In [17]:

# Normalizar nombres de columnas
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print("Normalización de nombres de columnas completada.")

# Filtrar planetas en zona habitable aproximada
def is_habitable(row):
    L = (row['st_teff'] / 5778)**4 * (row['st_rad'] / 1.0)**2
    inner_bound = np.sqrt(L / 1.1)  # Límite interno de zona habitable
    outer_bound = np.sqrt(L / 0.53)  # Límite externo de zona habitable
    return inner_bound <= row['pl_orbsmax'] <= outer_bound

df['habitable_zone'] = df.apply(is_habitable, axis=1)
df = df[df['habitable_zone']]
print("Filtrado de exoplanetas en zona habitable completado. Tamaño actual:", df.shape)

# Guardar dataset limpio
df.to_csv("cleaned_exoplanets.csv", index=False)
print("Preprocesamiento completado. Datos guardados en 'cleaned_exoplanets.csv'.")


Normalización de nombres de columnas completada.
Filtrado de exoplanetas en zona habitable completado. Tamaño actual: (15, 17)
Preprocesamiento completado. Datos guardados en 'cleaned_exoplanets.csv'.
